<a href="https://colab.research.google.com/github/Banner-19/VoiceForge/blob/main/AudFAKE_v4_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install necessary packages
!pip install wget torch numpy scipy soundfile librosa pyaudio webrtcvad

# Clone the VITS repository for vocoder usage
!git clone https://github.com/jaywalnut310/vits.git
!pip install -r vits/requirements.txt
!apt-get install -y espeak


  Using cached PyAudio-0.2.14.tar.gz (47 kB)
  Installing build dependencies ... canceled
ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 377, in run
    requirement_set = resolver.resolve(
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/resolution/resolvelib/resolver.py", line 92, in resolve
    result = self._result = resolver.resolve(
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/resolvelib/resolvers.py", line 546, in resolve
    state = resolution.resolve(requirements, max_rounds=max_rounds)
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/resolvelib/re

In [ ]:
# Install PortAudio library
!apt-get install -y portaudio19-dev

# Install PyAudio
!pip install pyaudio

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libportaudiocpp0
Suggested packages:
  portaudio19-doc
The following NEW packages will be installed:
  libportaudiocpp0 portaudio19-dev
0 upgraded, 2 newly installed, 0 to remove and 45 not upgraded.
Need to get 122 kB of archives.
After this operation, 703 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudiocpp0 amd64 19.6.0-1.1 [16.1 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 portaudio19-dev amd64 19.6.0-1.1 [106 kB]
Fetched 122 kB in 2s (75.1 kB/s)
Selecting previously unselected package libportaudiocpp0:amd64.
(Reading database ... 122243 files and directories currently installed.)
Preparing to unpack .../libportaudiocpp0_19.6.0-1.1_amd64.deb ...
Unpacking libportaudiocpp0:amd64 (19.6.0-1.1) ...
Selecting previously unselected package portaudio19-dev:

In [ ]:
import wget
import tarfile
import os

# Download and extract the LJSpeech dataset
#url = 'https://data.keithito.com/data/speech/LJSpeech-1.1.tar.bz2'
#wget.download(url)

#with tarfile.open('LJSpeech-1.1.tar.bz2', 'r:bz2') as tar:
#    tar.extractall()

# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import librosa
import soundfile as sf
from torch.utils.data import Dataset, DataLoader

import sys
import os
# Assuming vits is located in '/path/to/vits'
vits_path = '/content/vits'
sys.path.append(vits_path)

from vits.text import text_to_sequence
from vits.text.symbols import symbols

class LJSpeechDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = '/content/LJSpeech-1.1'
        self.transform = transform
        self.metadata = self._load_metadata()

    def _load_metadata(self):
        metadata_path = os.path.join(self.data_dir, 'metadata.csv')
        with open(metadata_path, 'r') as f:
            metadata = [line.strip().split('|') for line in f]
        return metadata

    def __len__(self):
        return len(self.metadata)

    def __getitem__(self, idx):
        wav_path = os.path.join(self.data_dir, 'wavs', self.metadata[idx][0] + '.wav')
        y, sr = sf.read(wav_path)

        # Resample y to 22050 Hz if sr is not already 22050
        if sr != 22050:
            y = librosa.resample(y, sr, 22050)
            sr = 22050

        y = y.astype(np.float32)
        text = self.metadata[idx][1]

        # Compute mel spectrogram
        mel_spectrogram = librosa.feature.melspectrogram(y, sr=sr, n_fft=1024, hop_length=256, n_mels=80)
        mel_spectrogram = librosa.power_to_db(mel_spectrogram, ref=np.max)

        # Convert to PyTorch tensors
        text_tensor = torch.tensor(text_to_sequence(text, ['english_cleaners']), dtype=torch.long)
        mel_tensor = torch.tensor(mel_spectrogram, dtype=torch.float32)

        return text_tensor, mel_tensor

# Create the dataset and data loader
dataset = LJSpeechDataset(data_dir='LJSpeech-1.1')
data_loader = DataLoader(dataset, batch_size=16, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))


In [ ]:
pip install phonemizer


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 54.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 24.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.4/213.4 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 47.5 MB/s eta 0:00:00
  Created wheel for pylatexenc: filename=pylatexenc-2.10-py3-none-any.whl size=136816 sha256=4da5e5b71869b3d2e23aa6f3d1102dd0caa489e2d36d83dedb1970bd64e5c72a
  Stored in directory: /root/.cache/pip/wheels/d3/31/8b/e09b0386afd80cfc556c00408c9aeea5c35c4d484a9c762fd5
Successfully built pylatexenc


In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(Seq2Seq, self).__init__()
        self.encoder = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.decoder = nn.LSTM(hidden_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        _, (hidden, cell) = self.encoder(x)
        outputs, _ = self.decoder(x, (hidden, cell))
        outputs = self.fc(outputs)
        return outputs

# Hyperparameters
input_dim = 128  # Size of text embeddings
hidden_dim = 256
output_dim = 80  # Number of mel-spectrogram bins

# Instantiate the model
model = Seq2Seq(input_dim, hidden_dim, output_dim).cuda()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
num_epochs = 50

for epoch in range(num_epochs):
    for texts, mels in data_loader:
        texts = [torch.tensor(t, dtype=torch.long) for t in texts]
        mels = [torch.tensor(m, dtype=torch.float32) for m in mels]

        texts = torch.nn.utils.rnn.pad_sequence(texts, batch_first=True).cuda()
        mels = torch.nn.utils.rnn.pad_sequence(mels, batch_first=True).cuda()

        optimizer.zero_grad()
        outputs = model(texts)
        loss = criterion(outputs, mels)
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


TypeError: melspectrogram() takes 0 positional arguments but 1 positional argument (and 3 keyword-only arguments) were given

In [ ]:
import torch
from waveglow.denoiser import Denoiser

# Load pre-trained WaveGlow model
waveglow = torch.hub.load('nvidia/DeepLearningExamples:torchhub', 'nvidia_waveglow')
waveglow = waveglow.remove_weightnorm(waveglow)
waveglow.eval().cuda()
denoiser = Denoiser(waveglow).cuda()

def mel_to_audio(mel):
    mel = torch.tensor(mel).unsqueeze(0).cuda()
    with torch.no_grad():
        audio = waveglow.infer(mel, sigma=0.666)
        audio = denoiser(audio, strength=0.01)[:, 0]
    return audio.cpu().numpy()

# Example usage
text = "Hello, how are you?"
text_seq = torch.tensor(text_to_sequence(text, ['english_cleaners']), dtype=torch.long).unsqueeze(0).cuda()
with torch.no_grad():
    mel = model(text_seq).cpu().numpy()
audio = mel_to_audio(mel)

# Save the synthesized audio to a file
sf.write('synthesized_output.wav', audio, 22050)

# Playback the audio
import IPython.display as ipd
ipd.Audio('synthesized_output.wav')
